# Hands-on session with pke

## Part 2 : model parameterization

As we have seen, keyphrase extraction is commonly treated as a three-stage process involving:
 1. identifying keyphrase candidates
 2. weighting these candidates
 3. selecting the N-best candidates as keyphrases
 
Each of the above stages can be parameterized/modified in `pke` and we will see how below.

In [4]:
import pke

# sample document (2040.abstr from the Hulth-2003 dataset)
sample = """Inverse problems for a mathematical model of ion exchange in a compressible ion exchanger.
A mathematical model of ion exchange is considered, allowing for ion exchanger compression in the process
of ion exchange. Two inverse problems are investigated for this model, unique solvability is proved, and
numerical solution methods are proposed. The efficiency of the proposed methods is demonstrated by a
numerical experiment."""

# normalize spacing (replace newlines with whitespaces)
sample = sample.replace("\n", " ")

### Stage-1 : candidate selection 

In [5]:
# let us start with a TopicRank model
extractor = pke.unsupervised.TopicRank()

# load the document using the initialized model
extractor.load_document(input=sample, language='en')

# by default the candidate